In [1]:
import itertools
import os

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
import re
from sklearn.metrics import accuracy_score, classification_report

import tensorflow

from numpy import loadtxt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from tensorflow.keras.utils import to_categorical
import statsmodels.api as sm

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Artem\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df1_dims = pd.read_csv("Dimensions_texts_Stories.csv")
df2_dims = pd.read_csv("Dimensions_texts_News.csv")
df3_dims = pd.read_csv("Dimensions_texts_Essays.csv")

df1_stats = pd.read_csv("Statistics_texts_Stories.csv")
df2_stats = pd.read_csv("Statistics_texts_News.csv")
df3_stats = pd.read_csv("Statistics_texts_Essays.csv")

In [3]:
df1_stats = df1_stats.drop(columns = ['Unnamed: 89'])
df2_stats = df2_stats.drop(columns = ['Unnamed: 86'])
df3_stats = df3_stats.drop(columns = ['Unnamed: 81'])

In [4]:
df1_stats

,Filename,Tokens,AWL,TTR,#,'',:,AMP,ANDC,CAUS,...,[SPIN],[SUAV],[THATD],[WHCL],[WHOBJ],[WHQU],[WHSUB],[WZPAST],[WZPRES],``
0,text_0,1124,4.14,207,0.00,0.00,0.36,0.27,0.09,0.09,...,0.00,0.36,0.71,0.18,0.00,0.00,0.00,0.00,0.27,0.00
1,text_10,5697,4.35,192,0.07,0.40,0.53,0.12,0.07,0.04,...,0.02,0.18,0.05,0.02,0.02,0.00,0.19,0.07,0.14,0.00
2,text_11,9038,3.72,156,0.00,2.66,0.45,0.45,0.03,0.15,...,0.00,0.41,0.91,0.23,0.03,0.20,0.11,0.12,0.00,0.00
3,text_12,19671,4.45,242,0.08,0.62,0.85,0.23,0.12,0.08,...,0.01,0.23,0.24,0.07,0.06,0.01,0.44,0.05,0.07,0.01
4,text_13,8300,4.02,168,0.00,3.70,0.45,0.57,0.13,0.17,...,0.00,0.58,0.55,0.13,0.01,0.29,0.12,0.07,0.00,0.00
5,text_14,12573,3.88,191,0.02,3.28,0.35,0.05,0.01,0.02,...,0.00,0.30,0.56,0.16,0.00,0.18,0.04,0.10,0.04,0.00
6,text_15,29520,4.47,194,0.00,0.25,0.11,0.47,0.18,0.14,...,0.01,0.37,0.19,0.14,0.19,0.00,0.48,0.04,0.02,0.00
7,text_1,23,3.57,13,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
8,text_2,157,4.00,96,0.00,0.00,1.91,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
9,text_3,1639,4.05,216,0.00,0.00,0.12,0.12,0.00,0.00,...,0.00,0.61,0.85,0.06,0.00,0.00,0.18,0.00,0.00,0.00


In [5]:
df1_dims

,Filename,Dimension1,Dimension2,Dimension3,Dimension4,Dimension5,Dimension6,Closest Text Type
0,text_0,-1.00,3.50,-2.12,-0.23,-0.03,-2.64,Imaginative narrative
1,text_10,-17.97,-5.86,4.86,-1.92,-1.19,-2.78,Learned exposition
2,text_11,13.26,-2.72,-0.52,-1.87,-2.77,-1.74,Involved persuasion
3,text_12,-14.15,-3.84,4.72,-0.96,-0.93,-1.18,Learned exposition
4,text_13,3.51,0.31,-0.87,-0.18,-2.12,-0.98,Involved persuasion
5,text_14,-2.56,-1.91,-1.97,-2.93,-2.88,-1.68,General narrative exposition
6,text_15,-2.85,-5.10,5.76,3.12,0.14,0.88,Involved persuasion
7,text_1,7.29,-7.38,-4.87,-9.27,-3.92,-3.50,Situated reportage
8,text_2,-0.89,-1.11,-2.45,0.66,-3.92,-1.97,Involved persuasion
9,text_3,-6.76,-0.46,-0.12,-1.35,-1.44,-1.85,General narrative exposition


In [6]:
df2_stats

,Filename,Tokens,AWL,TTR,'',:,AMP,ANDC,CAUS,CC,...,[SMP],[SPAU],[SPIN],[SUAV],[THATD],[WHCL],[WHOBJ],[WHSUB],[WZPAST],[WZPRES]
0,text_0,1141.00,4.51,187.00,0.09,0.18,0.26,0.00,0.18,2.02,...,0.09,0.35,0.18,0.18,0.96,0.00,0.09,0.61,0.18,0.18
1,text_10,928.00,4.69,190.00,0.00,0.65,0.00,0.11,0.32,2.05,...,0.00,0.11,0.11,0.65,0.75,0.00,0.11,0.86,0.00,0.00
2,text_11,948.00,4.36,223.00,0.11,0.53,0.11,0.00,0.21,2.43,...,0.00,0.00,0.21,0.21,0.32,0.00,0.00,0.53,0.00,0.00
3,text_12,902.00,4.45,196.00,0.00,0.00,0.00,0.00,0.11,1.55,...,0.00,0.11,0.00,0.44,1.00,0.00,0.00,0.00,0.00,0.22
4,text_13,476.00,4.44,177.00,0.00,0.42,0.00,0.00,0.00,1.47,...,0.21,0.00,0.00,0.42,0.00,0.00,0.00,0.42,0.21,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,text_6,908.00,4.75,211.00,0.11,0.44,0.11,0.33,0.11,2.09,...,0.00,0.66,0.00,0.00,0.44,0.00,0.11,0.55,0.00,0.11
65,text_7,1446.00,4.47,241.00,0.00,0.69,0.07,0.14,0.07,1.80,...,0.21,0.55,0.00,0.21,0.55,0.14,0.00,0.07,0.00,0.00
66,text_8,926.00,4.34,220.00,0.00,0.11,0.54,0.00,0.11,2.16,...,0.00,0.32,0.00,0.00,0.65,0.11,0.11,0.54,0.11,0.00
67,text_9,700.00,4.69,225.00,0.00,0.14,0.00,0.00,0.14,1.43,...,0.14,0.29,0.00,0.57,0.43,0.14,0.00,0.57,0.00,0.00


In [7]:
df3_stats

,Filename,Tokens,AWL,TTR,:,AMP,ANDC,CAUS,CC,CD,...,[SMP],[SPAU],[SPIN],[SUAV],[THATD],[WHCL],[WHOBJ],[WHSUB],[WZPAST],[WZPRES]
0,text_0,154.00,4.16,76.00,0.0,0.00,0.00,0.00,3.90,0.65,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,text_100,150.00,4.50,70.00,0.0,0.00,0.00,0.00,6.67,0.00,...,0.00,0.67,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.67
2,text_101,316.00,4.22,155.00,0.0,0.95,0.32,0.00,1.90,0.32,...,0.00,0.32,0.00,0.00,0.63,0.00,0.00,0.63,0.00,0.00
3,text_102,205.00,4.42,99.00,0.0,0.49,0.00,0.00,1.46,1.95,...,0.00,0.00,0.00,0.49,0.49,0.00,0.00,1.46,0.00,0.00
4,text_103,377.00,4.69,148.00,0.0,0.53,0.00,0.27,2.92,1.33,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.53,0.27,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
450,text_97,126.00,4.92,65.00,0.0,0.00,0.00,0.79,2.38,0.79,...,0.00,0.00,0.00,0.00,0.00,0.00,0.79,1.59,0.00,0.00
451,text_98,163.00,4.36,76.00,0.0,0.00,0.00,0.00,1.84,0.61,...,0.00,0.00,0.00,0.61,0.00,0.00,0.00,0.00,0.61,0.00
452,text_99,247.00,4.63,114.00,0.0,0.00,0.81,0.81,1.21,0.81,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.40
453,text_9,315.00,4.66,147.00,0.0,0.00,0.00,1.59,1.27,0.00,...,0.00,0.63,0.00,0.95,0.00,0.00,0.00,0.32,0.00,0.00


## Features (количественные характеристики): 

* Tokens – количество токенов.
* AWL – средняя длина слова.
* TTR – количество тренировочных токенов (400).
* AMP – наличие слов-усилителей.
* ANDC – управление независимой клаузой.
* CAUS – наличие обстоятельств причины по типу «because».
* CC – ???.
* CD – кардинальное число (элементы множества).
* CONC – концессионная обусловленность (наличие слов типа «although», «though» и т.п.)
* COND – слова состояния, условности (наличие слов типа «if», «unless» и т.п.)
* CONJ – слова-союзы.
* DEMO – указательные местоимения, которые не подходят ни к одному из следующих тэгов: DEMP, TOBJ, TSUB, THAC или THVC.
* DEMP – указательное местоимение.
* DT – слово-определитель.
* DWNT – слова, обозначающие незавершённость действия («almost», «hardly», «somewhat», «» и многие другие).
* EMPH – слова с эмоциональным оттенком («really», «just», «for sure» и т.п.)
* EX – слова с указанием существования чего-либо («there»).
* FPP1 – местоимения первого лица («I», «we», «our», «my», «ourselves», «us», «me», «myself»).
* FW – ???.
* GER – герундий (глаголы с «-ing» на конце).
* HDG – показатели неопределённости («more or less», «maybe», «something like» и т.п.)
* IN – подчинительные союзы.
* INPR – местоимения без точного указания на конкретное лицо («everyone», «anyone», «someone», «something» и т.п.)
* JJ – прилагательные.
* NEMD – глаголы обязательства («ought», «should», «must»).
* NN – существительные.
* NOMZ – отглагольные существительные («-tion», «-ness», «-ment» и т.п).
* OSUB – слова по типу «since», «while», «whilst», «whereupon», «whereas», «whereby», «such that», «so that».
* PDT – слова, стоящие перед определением.
* PHC – союз «и».
* PIN – предлоги.
* PIT – местоимение «it».
* PLACE – указатели места («above», «below», «», «» и т.п.).
* POMD – модальные глаголы возможности («can», «may», «might», «could»).
* PRED – предикативное прилагательное.
* PRMD – глаголы с указанием на будущее время («will», «shall», «would»).
* PRP – личное местоимение.
* QUAN – числительное.
* QUPR – числительное местоимение.
* RB – наречие.
* RP – ???.
* SPP2 – местоимения в форме второго лица («you», «your», «yourself», «yourselves», «thy», «thee», «thyself», «thou»).
* SYM – ???.
* SYNE – конструкции «no» + прилагательное.
* THAC – конструкции «that» + прилагательное.
* THVC – конструкции «that» + глагол.
* TIME – времени («afterwards», «again», «earlier», «early» и другие).
* TO – инфинитивы, которые следуют после слова «to».
* TOBJ – конструкции по типу «that» + подлежащее + сказуемое; применяется в отношении объекта предложения.
* TPP3 – местоимения в форме третьего лица («she», «he», «they», «her», «him», «them», «his», «their», «himself», «herself», «themselves»).
* TSUB – конструкции по типу «that» + подлежащее + сказуемое; применяется в отношении субъекта предложения.
* UH – ???.
* VB – глаголы.
* VBD – глаголы в форме прошедшего времени.
* VBG – глаголы с формой Present Continuous с «-ing» на конце.
* VBN – глаголы с формой Past Continuous с «-ing» на конце.
* VPRT – глаголы с формой Present Simple.
* WDT – ???.
* WP – местоимения, начинающиеся на «wh-».
* WPS – местоимения, начинающиеся на «wh-» и имеющие посессивный характер.
* XX0 – отрицания; слова «not» и «n’t».
* [BEMA] – глагольная фраза с «be».
* [BYPA] – фразы в пассивном залоге с «by».
* [PASS] – фразы в пассивном залоге без пациенса.
* [PEAS] – фразы в перфекте («have» + глагол).
* [PIRE] – предикаты сложноподчинённого предложения со словами «who», «whose» или «which»; причём эти слова обозначают субъект из основной части предложения.
* [PRIV] – список глаголов по Quirk et al. (1985: 1181–2): «feel», «recall», «accept», «assume», «prove», «believe», «remember», «calculate» и т.п., в т.ч. их словоформы.
* [PROD] – использования слова «do» в качестве самостоятельного глагола.
* [PUBV] – глаголы по типу «announce», «certify», «say», «write» и т.п.
* [SMP] – глаголы «seem» и «appear».
* [SPAU] – конструкции типа «вспомогательный глагол + минимум одно наречие + базовая форма глагола».
* [SPIN] – конструкции типа «to + минимум одно наречие + базовая форма глагола».
* [SUAV] – список глаголов убеждения по Quirk et al. (1985: 1182–3): «ensure», «insist», «order», «propose», «request», «suggest» и т.п.
* [THATD] – подчинительные клаузы без «that».
* [WHCL] – клаузы с вопросительными словами на «wh-», после которых следует слово, не являющееся вспомогательной частью речи.
* [WHOBJ] – определённые придаточные предложения, имеющие вопросительные слова на «wh-» в качестве объекта.
* [WHSUB] – определённые придаточные предложения, имеющие вопросительные слова на «wh-» в качестве субъекта.
* [WZPAST] – фразы по типу «The solution produced by this process»
* [WZPRES] – фразы по типу «the event causing this decline is…»

In [8]:
y_df1 = df1_dims['Closest Text Type']
y_df2 = df2_dims['Closest Text Type']
y_df3 = df3_dims['Closest Text Type']

In [9]:
y_df1.unique()

array(['Imaginative narrative', 'Learned exposition',
       'Involved persuasion', 'General narrative exposition',
       'Situated reportage'], dtype=object)

In [10]:
y_df2.unique()

array(['General narrative exposition', 'Learned exposition',
       'Involved persuasion', 'Situated reportage'], dtype=object)

In [11]:
y_df3.unique()

array(['General narrative exposition', 'Involved persuasion',
       'Learned exposition', 'Scientific exposition',
       'Informational interaction', 'Situated reportage'], dtype=object)

In [12]:
df4 = list(list(y_df1) + list(y_df2) + list(y_df3))
len(df4)

540

In [15]:
bads = []

for l in list(df1_stats.columns):
    if re.findall('[a-zA-Z]', l) == []:
        bads.append(l)

for l in list(df2_stats.columns):
    if re.findall('[a-zA-Z]', l) == []:
        bads.append(l)

for l in list(df3_stats.columns):
    if re.findall('[a-zA-Z]', l) == []:
        bads.append(l)

In [16]:
set(bads)

{'#', "''", ':', '``'}

In [18]:
for l in list(df1_stats.columns):
    if l not in df2_stats.columns and l not in df3_stats.columns:
        print(l)

df1_stats = df1_stats.drop(["POS", "[WHQU]"], axis = 1)

#
POS
[WHQU]
``


In [19]:
for l in list(df1_stats.columns):
    if l not in df3_stats.columns:
        print(l)

#
''
DPAR
[PASTP]
[SERE]
``


In [20]:
for l in list(df2_stats.columns):
    if l not in df3_stats.columns:
        print(l)

''
DPAR
LS
[PASTP]
[SERE]


In [21]:
for l in list(df2_stats.columns):
    if l not in df1_stats.columns and l not in df3_stats.columns:
        print(l)

LS


In [22]:
df2_stats.columns

Index(['Filename', 'Tokens', 'AWL', 'TTR', '''', ':', 'AMP', 'ANDC', 'CAUS',
       'CC', 'CD', 'CONC', 'COND', 'CONJ', 'DEMO', 'DEMP', 'DPAR', 'DT',
       'DWNT', 'EMPH', 'EX', 'FPP1', 'FW', 'GER', 'HDG', 'IN', 'INPR', 'JJ',
       'LS', 'NEMD', 'NN', 'NOMZ', 'OSUB', 'PDT', 'PHC', 'PIN', 'PIT', 'PLACE',
       'POMD', 'PRED', 'PRMD', 'PRP', 'QUAN', 'QUPR', 'RB', 'RP', 'SPP2',
       'SYM', 'SYNE', 'THAC', 'THVC', 'TIME', 'TO', 'TOBJ', 'TPP3', 'TSUB',
       'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VPRT', 'WDT', 'WP', 'WPS', 'XX0',
       '[BEMA]', '[BYPA]', '[PASS]', '[PASTP]', '[PEAS]', '[PIRE]', '[PRIV]',
       '[PROD]', '[PUBV]', '[SERE]', '[SMP]', '[SPAU]', '[SPIN]', '[SUAV]',
       '[THATD]', '[WHCL]', '[WHOBJ]', '[WHSUB]', '[WZPAST]', '[WZPRES]'],
      dtype='object')

In [23]:
df1_stats = df1_stats.drop(["DPAR", "[PASTP]", "[SERE]"], axis = 1)
df2_stats = df2_stats.drop(["DPAR", "[PASTP]", "[SERE]"], axis = 1)
df2_stats = df2_stats.drop(["LS"], axis = 1)

In [24]:
for l in list(df1_stats.columns):
    if l not in df3_stats.columns:
        print(l)


#
''
``


In [25]:
df1_stats = df1_stats.drop(["#", "''", ":","``"], axis = 1)

In [26]:
df2_stats = df2_stats.drop(["''", ":"], axis = 1)

In [27]:
df3_stats = df3_stats.drop([":"], axis = 1)

In [28]:
print(len(df1_stats.columns))
print(len(df2_stats.columns))
print(len(df3_stats.columns))

80
80
80


In [29]:
df1_stats.columns

Index(['Filename', 'Tokens', 'AWL', 'TTR', 'AMP', 'ANDC', 'CAUS', 'CC', 'CD',
       'CONC', 'COND', 'CONJ', 'DEMO', 'DEMP', 'DT', 'DWNT', 'EMPH', 'EX',
       'FPP1', 'FW', 'GER', 'HDG', 'IN', 'INPR', 'JJ', 'NEMD', 'NN', 'NOMZ',
       'OSUB', 'PDT', 'PHC', 'PIN', 'PIT', 'PLACE', 'POMD', 'PRED', 'PRMD',
       'PRP', 'QUAN', 'QUPR', 'RB', 'RP', 'SPP2', 'SYM', 'SYNE', 'THAC',
       'THVC', 'TIME', 'TO', 'TOBJ', 'TPP3', 'TSUB', 'UH', 'VB', 'VBD', 'VBG',
       'VBN', 'VPRT', 'WDT', 'WP', 'WPS', 'XX0', '[BEMA]', '[BYPA]', '[PASS]',
       '[PEAS]', '[PIRE]', '[PRIV]', '[PROD]', '[PUBV]', '[SMP]', '[SPAU]',
       '[SPIN]', '[SUAV]', '[THATD]', '[WHCL]', '[WHOBJ]', '[WHSUB]',
       '[WZPAST]', '[WZPRES]'],
      dtype='object')

In [30]:
df2_stats.columns

Index(['Filename', 'Tokens', 'AWL', 'TTR', 'AMP', 'ANDC', 'CAUS', 'CC', 'CD',
       'CONC', 'COND', 'CONJ', 'DEMO', 'DEMP', 'DT', 'DWNT', 'EMPH', 'EX',
       'FPP1', 'FW', 'GER', 'HDG', 'IN', 'INPR', 'JJ', 'NEMD', 'NN', 'NOMZ',
       'OSUB', 'PDT', 'PHC', 'PIN', 'PIT', 'PLACE', 'POMD', 'PRED', 'PRMD',
       'PRP', 'QUAN', 'QUPR', 'RB', 'RP', 'SPP2', 'SYM', 'SYNE', 'THAC',
       'THVC', 'TIME', 'TO', 'TOBJ', 'TPP3', 'TSUB', 'UH', 'VB', 'VBD', 'VBG',
       'VBN', 'VPRT', 'WDT', 'WP', 'WPS', 'XX0', '[BEMA]', '[BYPA]', '[PASS]',
       '[PEAS]', '[PIRE]', '[PRIV]', '[PROD]', '[PUBV]', '[SMP]', '[SPAU]',
       '[SPIN]', '[SUAV]', '[THATD]', '[WHCL]', '[WHOBJ]', '[WHSUB]',
       '[WZPAST]', '[WZPRES]'],
      dtype='object')

In [31]:
df3_stats.columns

Index(['Filename', 'Tokens', 'AWL', 'TTR', 'AMP', 'ANDC', 'CAUS', 'CC', 'CD',
       'CONC', 'COND', 'CONJ', 'DEMO', 'DEMP', 'DT', 'DWNT', 'EMPH', 'EX',
       'FPP1', 'FW', 'GER', 'HDG', 'IN', 'INPR', 'JJ', 'NEMD', 'NN', 'NOMZ',
       'OSUB', 'PDT', 'PHC', 'PIN', 'PIT', 'PLACE', 'POMD', 'PRED', 'PRMD',
       'PRP', 'QUAN', 'QUPR', 'RB', 'RP', 'SPP2', 'SYM', 'SYNE', 'THAC',
       'THVC', 'TIME', 'TO', 'TOBJ', 'TPP3', 'TSUB', 'UH', 'VB', 'VBD', 'VBG',
       'VBN', 'VPRT', 'WDT', 'WP', 'WPS', 'XX0', '[BEMA]', '[BYPA]', '[PASS]',
       '[PEAS]', '[PIRE]', '[PRIV]', '[PROD]', '[PUBV]', '[SMP]', '[SPAU]',
       '[SPIN]', '[SUAV]', '[THATD]', '[WHCL]', '[WHOBJ]', '[WHSUB]',
       '[WZPAST]', '[WZPRES]'],
      dtype='object')

In [32]:
df1_stats['Genre'] = "Stories"
df2_stats['Genre'] = "News"
df3_stats['Genre'] = "Essays"

In [33]:
df1_stats

,Filename,Tokens,AWL,TTR,AMP,ANDC,CAUS,CC,CD,CONC,...,[SPAU],[SPIN],[SUAV],[THATD],[WHCL],[WHOBJ],[WHSUB],[WZPAST],[WZPRES],Genre
0,text_0,1124,4.14,207,0.27,0.09,0.09,2.22,0.53,0.18,...,0.18,0.00,0.36,0.71,0.18,0.00,0.00,0.00,0.27,Stories
1,text_10,5697,4.35,192,0.12,0.07,0.04,3.05,0.61,0.09,...,0.30,0.02,0.18,0.05,0.02,0.02,0.19,0.07,0.14,Stories
2,text_11,9038,3.72,156,0.45,0.03,0.15,2.66,0.81,0.01,...,0.18,0.00,0.41,0.91,0.23,0.03,0.11,0.12,0.00,Stories
3,text_12,19671,4.45,242,0.23,0.12,0.08,2.06,0.98,0.09,...,0.25,0.01,0.23,0.24,0.07,0.06,0.44,0.05,0.07,Stories
4,text_13,8300,4.02,168,0.57,0.13,0.17,2.88,0.48,0.06,...,0.28,0.00,0.58,0.55,0.13,0.01,0.12,0.07,0.00,Stories
5,text_14,12573,3.88,191,0.05,0.01,0.02,1.48,0.68,0.02,...,0.17,0.00,0.30,0.56,0.16,0.00,0.04,0.10,0.04,Stories
6,text_15,29520,4.47,194,0.47,0.18,0.14,3.00,0.88,0.13,...,0.51,0.01,0.37,0.19,0.14,0.19,0.48,0.04,0.02,Stories
7,text_1,23,3.57,13,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Stories
8,text_2,157,4.00,96,0.00,0.00,0.00,5.10,2.55,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Stories
9,text_3,1639,4.05,216,0.12,0.00,0.00,2.20,0.67,0.00,...,0.00,0.00,0.61,0.85,0.06,0.00,0.18,0.00,0.00,Stories


In [34]:
files_1 = []
for f in df1_stats['Filename']:
    files_1.append('stories_' + f)

files_2 = []
for f in df2_stats['Filename']:
    files_2.append('news_' + f)

files_3 = []
for f in df3_stats['Filename']:
    files_3.append('essays_' + f)

In [35]:
df1_stats = df1_stats.assign(Filename = lambda x: ('stories_' + x['Filename']))
df2_stats = df2_stats.assign(Filename = lambda x: ('news_' + x['Filename']))
df3_stats = df3_stats.assign(Filename = lambda x: ('essays_' + x['Filename']))

In [36]:
df3_stats

,Filename,Tokens,AWL,TTR,AMP,ANDC,CAUS,CC,CD,CONC,...,[SPAU],[SPIN],[SUAV],[THATD],[WHCL],[WHOBJ],[WHSUB],[WZPAST],[WZPRES],Genre
0,essays_text_0,154.00,4.16,76.00,0.00,0.00,0.00,3.90,0.65,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Essays
1,essays_text_100,150.00,4.50,70.00,0.00,0.00,0.00,6.67,0.00,0.67,...,0.67,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.67,Essays
2,essays_text_101,316.00,4.22,155.00,0.95,0.32,0.00,1.90,0.32,0.32,...,0.32,0.00,0.00,0.63,0.00,0.00,0.63,0.00,0.00,Essays
3,essays_text_102,205.00,4.42,99.00,0.49,0.00,0.00,1.46,1.95,0.00,...,0.00,0.00,0.49,0.49,0.00,0.00,1.46,0.00,0.00,Essays
4,essays_text_103,377.00,4.69,148.00,0.53,0.00,0.27,2.92,1.33,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.53,0.27,0.00,Essays
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
450,essays_text_97,126.00,4.92,65.00,0.00,0.00,0.79,2.38,0.79,0.00,...,0.00,0.00,0.00,0.00,0.00,0.79,1.59,0.00,0.00,Essays
451,essays_text_98,163.00,4.36,76.00,0.00,0.00,0.00,1.84,0.61,0.00,...,0.00,0.00,0.61,0.00,0.00,0.00,0.00,0.61,0.00,Essays
452,essays_text_99,247.00,4.63,114.00,0.00,0.81,0.81,1.21,0.81,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.40,Essays
453,essays_text_9,315.00,4.66,147.00,0.00,0.00,1.59,1.27,0.00,0.00,...,0.63,0.00,0.95,0.00,0.00,0.00,0.32,0.00,0.00,Essays


In [37]:
df3_dims

,Filename,Dimension1,Dimension2,Dimension3,Dimension4,Dimension5,Dimension6,Closest Text Type
0,text_0,-12.91,-7.38,-2.80,-0.76,0.14,1.15,General narrative exposition
1,text_100,-5.60,-7.08,0.17,-5.39,-3.92,1.26,General narrative exposition
2,text_101,6.16,-5.22,3.08,3.29,0.99,2.43,Involved persuasion
3,text_102,-0.32,-6.25,10.92,-1.37,6.29,8.45,Involved persuasion
4,text_103,1.10,-4.23,2.52,3.96,5.16,6.62,Involved persuasion
...,...,...,...,...,...,...,...,...
450,text_97,2.17,-4.91,8.34,-6.43,1.02,-0.77,Involved persuasion
451,text_98,-11.51,-4.75,4.16,-6.21,1.86,-3.50,Scientific exposition
452,text_99,2.72,-3.02,-2.26,5.76,1.83,2.18,Involved persuasion
453,text_9,1.75,-3.08,8.66,7.05,1.83,2.46,Involved persuasion


In [38]:
#df3_stats = df3_stats.drop(df3_stats[df3_stats['Filename'] == 'essays_texts'].index)

In [39]:
#df2_stats = df2_stats.drop(df2_stats[df2_stats['Filename'] == 'news_texts'].index)

In [40]:
df2_stats

,Filename,Tokens,AWL,TTR,AMP,ANDC,CAUS,CC,CD,CONC,...,[SPAU],[SPIN],[SUAV],[THATD],[WHCL],[WHOBJ],[WHSUB],[WZPAST],[WZPRES],Genre
0,news_text_0,1141.00,4.51,187.00,0.26,0.00,0.18,2.02,0.61,0.00,...,0.35,0.18,0.18,0.96,0.00,0.09,0.61,0.18,0.18,News
1,news_text_10,928.00,4.69,190.00,0.00,0.11,0.32,2.05,0.43,0.00,...,0.11,0.11,0.65,0.75,0.00,0.11,0.86,0.00,0.00,News
2,news_text_11,948.00,4.36,223.00,0.11,0.00,0.21,2.43,0.95,0.00,...,0.00,0.21,0.21,0.32,0.00,0.00,0.53,0.00,0.00,News
3,news_text_12,902.00,4.45,196.00,0.00,0.00,0.11,1.55,0.55,0.00,...,0.11,0.00,0.44,1.00,0.00,0.00,0.00,0.00,0.22,News
4,news_text_13,476.00,4.44,177.00,0.00,0.00,0.00,1.47,1.05,0.00,...,0.00,0.00,0.42,0.00,0.00,0.00,0.42,0.21,0.00,News
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,news_text_6,908.00,4.75,211.00,0.11,0.33,0.11,2.09,0.33,0.11,...,0.66,0.00,0.00,0.44,0.00,0.11,0.55,0.00,0.11,News
65,news_text_7,1446.00,4.47,241.00,0.07,0.14,0.07,1.80,0.83,0.14,...,0.55,0.00,0.21,0.55,0.14,0.00,0.07,0.00,0.00,News
66,news_text_8,926.00,4.34,220.00,0.54,0.00,0.11,2.16,1.08,0.22,...,0.32,0.00,0.00,0.65,0.11,0.11,0.54,0.11,0.00,News
67,news_text_9,700.00,4.69,225.00,0.00,0.00,0.14,1.43,0.71,0.14,...,0.29,0.00,0.57,0.43,0.14,0.00,0.57,0.00,0.00,News


In [41]:
df4_stats = pd.concat([df1_stats, df2_stats, df3_stats], axis=0)
y_df4 = pd.concat([y_df1, y_df2, y_df3], axis = 0)

In [42]:
y_df3

0      General narrative exposition
1      General narrative exposition
2               Involved persuasion
3               Involved persuasion
4               Involved persuasion
                   ...             
450             Involved persuasion
451           Scientific exposition
452             Involved persuasion
453             Involved persuasion
454             Involved persuasion
Name: Closest Text Type, Length: 455, dtype: object

In [43]:
#y_df4
df4_stats['Closest Text Type'] = y_df4

In [44]:
df4_stats

,Filename,Tokens,AWL,TTR,AMP,ANDC,CAUS,CC,CD,CONC,...,[SPIN],[SUAV],[THATD],[WHCL],[WHOBJ],[WHSUB],[WZPAST],[WZPRES],Genre,Closest Text Type
0,stories_text_0,1124.00,4.14,207.00,0.27,0.09,0.09,2.22,0.53,0.18,...,0.00,0.36,0.71,0.18,0.00,0.00,0.00,0.27,Stories,Imaginative narrative
1,stories_text_10,5697.00,4.35,192.00,0.12,0.07,0.04,3.05,0.61,0.09,...,0.02,0.18,0.05,0.02,0.02,0.19,0.07,0.14,Stories,Learned exposition
2,stories_text_11,9038.00,3.72,156.00,0.45,0.03,0.15,2.66,0.81,0.01,...,0.00,0.41,0.91,0.23,0.03,0.11,0.12,0.00,Stories,Involved persuasion
3,stories_text_12,19671.00,4.45,242.00,0.23,0.12,0.08,2.06,0.98,0.09,...,0.01,0.23,0.24,0.07,0.06,0.44,0.05,0.07,Stories,Learned exposition
4,stories_text_13,8300.00,4.02,168.00,0.57,0.13,0.17,2.88,0.48,0.06,...,0.00,0.58,0.55,0.13,0.01,0.12,0.07,0.00,Stories,Involved persuasion
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
450,essays_text_97,126.00,4.92,65.00,0.00,0.00,0.79,2.38,0.79,0.00,...,0.00,0.00,0.00,0.00,0.79,1.59,0.00,0.00,Essays,Involved persuasion
451,essays_text_98,163.00,4.36,76.00,0.00,0.00,0.00,1.84,0.61,0.00,...,0.00,0.61,0.00,0.00,0.00,0.00,0.61,0.00,Essays,Scientific exposition
452,essays_text_99,247.00,4.63,114.00,0.00,0.81,0.81,1.21,0.81,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.40,Essays,Involved persuasion
453,essays_text_9,315.00,4.66,147.00,0.00,0.00,1.59,1.27,0.00,0.00,...,0.00,0.95,0.00,0.00,0.00,0.32,0.00,0.00,Essays,Involved persuasion


In [45]:
#df4_dims = pd.concat([df1_dims, df2_dims, df3_dims],axis = 1)

In [46]:
df4_stats_stor = df4_stats[df4_stats['Genre'] == 'Stories']
df4_stats_news = df4_stats[df4_stats['Genre'] == 'News']
df4_stats_essays = df4_stats[df4_stats['Genre'] == 'Essays']

In [47]:
df1_dims

,Filename,Dimension1,Dimension2,Dimension3,Dimension4,Dimension5,Dimension6,Closest Text Type
0,text_0,-1.00,3.50,-2.12,-0.23,-0.03,-2.64,Imaginative narrative
1,text_10,-17.97,-5.86,4.86,-1.92,-1.19,-2.78,Learned exposition
2,text_11,13.26,-2.72,-0.52,-1.87,-2.77,-1.74,Involved persuasion
3,text_12,-14.15,-3.84,4.72,-0.96,-0.93,-1.18,Learned exposition
4,text_13,3.51,0.31,-0.87,-0.18,-2.12,-0.98,Involved persuasion
5,text_14,-2.56,-1.91,-1.97,-2.93,-2.88,-1.68,General narrative exposition
6,text_15,-2.85,-5.10,5.76,3.12,0.14,0.88,Involved persuasion
7,text_1,7.29,-7.38,-4.87,-9.27,-3.92,-3.50,Situated reportage
8,text_2,-0.89,-1.11,-2.45,0.66,-3.92,-1.97,Involved persuasion
9,text_3,-6.76,-0.46,-0.12,-1.35,-1.44,-1.85,General narrative exposition


In [48]:
df4_stats_stor['Dimension1'] = df1_dims['Dimension1']
df4_stats_stor['Dimension2'] = df1_dims['Dimension2']
df4_stats_stor['Dimension3'] = df1_dims['Dimension3']
df4_stats_stor['Dimension4'] = df1_dims['Dimension4']
df4_stats_stor['Dimension5'] = df1_dims['Dimension5']
df4_stats_stor['Dimension6'] = df1_dims['Dimension6']

C:\Users\Artem\AppData\Local\Temp\ipykernel_26476\2592070386.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4_stats_stor['Dimension1'] = df1_dims['Dimension1']
C:\Users\Artem\AppData\Local\Temp\ipykernel_26476\2592070386.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4_stats_stor['Dimension2'] = df1_dims['Dimension2']
C:\Users\Artem\AppData\Local\Temp\ipykernel_26476\2592070386.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

In [49]:
df4_stats_news['Dimension1'] = df2_dims['Dimension1']
df4_stats_news['Dimension2'] = df2_dims['Dimension2']
df4_stats_news['Dimension3'] = df2_dims['Dimension3']
df4_stats_news['Dimension4'] = df2_dims['Dimension4']
df4_stats_news['Dimension5'] = df2_dims['Dimension5']
df4_stats_news['Dimension6'] = df2_dims['Dimension6']

C:\Users\Artem\AppData\Local\Temp\ipykernel_26476\2803066045.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4_stats_news['Dimension1'] = df2_dims['Dimension1']
C:\Users\Artem\AppData\Local\Temp\ipykernel_26476\2803066045.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4_stats_news['Dimension2'] = df2_dims['Dimension2']
C:\Users\Artem\AppData\Local\Temp\ipykernel_26476\2803066045.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

In [50]:
df4_stats_essays['Dimension1'] = df3_dims['Dimension1']
df4_stats_essays['Dimension2'] = df3_dims['Dimension2']
df4_stats_essays['Dimension3'] = df3_dims['Dimension3']
df4_stats_essays['Dimension4'] = df3_dims['Dimension4']
df4_stats_essays['Dimension5'] = df3_dims['Dimension5']
df4_stats_essays['Dimension6'] = df3_dims['Dimension6']

C:\Users\Artem\AppData\Local\Temp\ipykernel_26476\2034564425.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4_stats_essays['Dimension1'] = df3_dims['Dimension1']
C:\Users\Artem\AppData\Local\Temp\ipykernel_26476\2034564425.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4_stats_essays['Dimension2'] = df3_dims['Dimension2']
C:\Users\Artem\AppData\Local\Temp\ipykernel_26476\2034564425.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[

In [51]:
df4_stats_stor

,Filename,Tokens,AWL,TTR,AMP,ANDC,CAUS,CC,CD,CONC,...,[WZPAST],[WZPRES],Genre,Closest Text Type,Dimension1,Dimension2,Dimension3,Dimension4,Dimension5,Dimension6
0,stories_text_0,1124.0,4.14,207.0,0.27,0.09,0.09,2.22,0.53,0.18,...,0.00,0.27,Stories,Imaginative narrative,-1.00,3.50,-2.12,-0.23,-0.03,-2.64
1,stories_text_10,5697.0,4.35,192.0,0.12,0.07,0.04,3.05,0.61,0.09,...,0.07,0.14,Stories,Learned exposition,-17.97,-5.86,4.86,-1.92,-1.19,-2.78
2,stories_text_11,9038.0,3.72,156.0,0.45,0.03,0.15,2.66,0.81,0.01,...,0.12,0.00,Stories,Involved persuasion,13.26,-2.72,-0.52,-1.87,-2.77,-1.74
3,stories_text_12,19671.0,4.45,242.0,0.23,0.12,0.08,2.06,0.98,0.09,...,0.05,0.07,Stories,Learned exposition,-14.15,-3.84,4.72,-0.96,-0.93,-1.18
4,stories_text_13,8300.0,4.02,168.0,0.57,0.13,0.17,2.88,0.48,0.06,...,0.07,0.00,Stories,Involved persuasion,3.51,0.31,-0.87,-0.18,-2.12,-0.98
5,stories_text_14,12573.0,3.88,191.0,0.05,0.01,0.02,1.48,0.68,0.02,...,0.10,0.04,Stories,General narrative exposition,-2.56,-1.91,-1.97,-2.93,-2.88,-1.68
6,stories_text_15,29520.0,4.47,194.0,0.47,0.18,0.14,3.00,0.88,0.13,...,0.04,0.02,Stories,Involved persuasion,-2.85,-5.10,5.76,3.12,0.14,0.88
7,stories_text_1,23.0,3.57,13.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,Stories,Situated reportage,7.29,-7.38,-4.87,-9.27,-3.92,-3.50
8,stories_text_2,157.0,4.00,96.0,0.00,0.00,0.00,5.10,2.55,0.00,...,0.00,0.00,Stories,Involved persuasion,-0.89,-1.11,-2.45,0.66,-3.92,-1.97
9,stories_text_3,1639.0,4.05,216.0,0.12,0.00,0.00,2.20,0.67,0.00,...,0.00,0.00,Stories,General narrative exposition,-6.76,-0.46,-0.12,-1.35,-1.44,-1.85


In [52]:
df4_stats_news

,Filename,Tokens,AWL,TTR,AMP,ANDC,CAUS,CC,CD,CONC,...,[WZPAST],[WZPRES],Genre,Closest Text Type,Dimension1,Dimension2,Dimension3,Dimension4,Dimension5,Dimension6
0,news_text_0,1141.00,4.51,187.00,0.26,0.00,0.18,2.02,0.61,0.00,...,0.18,0.18,News,General narrative exposition,-10.78,-0.52,3.45,-3.73,-2.78,1.07
1,news_text_10,928.00,4.69,190.00,0.00,0.11,0.32,2.05,0.43,0.00,...,0.00,0.00,News,Learned exposition,-18.57,-3.55,7.32,-0.67,1.15,-2.48
2,news_text_11,948.00,4.36,223.00,0.11,0.00,0.21,2.43,0.95,0.00,...,0.00,0.00,News,General narrative exposition,-13.58,-1.15,3.86,-3.37,-1.30,-1.86
3,news_text_12,902.00,4.45,196.00,0.00,0.00,0.11,1.55,0.55,0.00,...,0.00,0.22,News,General narrative exposition,-14.15,-0.88,4.76,-0.96,-3.92,0.93
4,news_text_13,476.00,4.44,177.00,0.00,0.00,0.00,1.47,1.05,0.00,...,0.21,0.00,News,General narrative exposition,-11.03,-4.12,2.35,-0.70,1.89,-2.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,news_text_6,908.00,4.75,211.00,0.11,0.33,0.11,2.09,0.33,0.11,...,0.00,0.11,News,Learned exposition,-14.48,-1.90,6.75,-3.70,0.77,-0.03
65,news_text_7,1446.00,4.47,241.00,0.07,0.14,0.07,1.80,0.83,0.14,...,0.00,0.00,News,General narrative exposition,-10.00,-3.94,0.88,-1.45,-0.04,-1.45
66,news_text_8,926.00,4.34,220.00,0.54,0.00,0.11,2.16,1.08,0.22,...,0.11,0.00,News,General narrative exposition,-7.96,-1.52,5.28,-5.00,-1.39,-1.31
67,news_text_9,700.00,4.69,225.00,0.00,0.00,0.14,1.43,0.71,0.14,...,0.00,0.00,News,Learned exposition,-14.17,-2.74,3.93,-0.16,1.99,-1.45


In [53]:
df4_stats_essays

,Filename,Tokens,AWL,TTR,AMP,ANDC,CAUS,CC,CD,CONC,...,[WZPAST],[WZPRES],Genre,Closest Text Type,Dimension1,Dimension2,Dimension3,Dimension4,Dimension5,Dimension6
0,essays_text_0,154.00,4.16,76.00,0.00,0.00,0.00,3.90,0.65,0.00,...,0.00,0.00,Essays,General narrative exposition,-12.91,-7.38,-2.80,-0.76,0.14,1.15
1,essays_text_100,150.00,4.50,70.00,0.00,0.00,0.00,6.67,0.00,0.67,...,0.00,0.67,Essays,General narrative exposition,-5.60,-7.08,0.17,-5.39,-3.92,1.26
2,essays_text_101,316.00,4.22,155.00,0.95,0.32,0.00,1.90,0.32,0.32,...,0.00,0.00,Essays,Involved persuasion,6.16,-5.22,3.08,3.29,0.99,2.43
3,essays_text_102,205.00,4.42,99.00,0.49,0.00,0.00,1.46,1.95,0.00,...,0.00,0.00,Essays,Involved persuasion,-0.32,-6.25,10.92,-1.37,6.29,8.45
4,essays_text_103,377.00,4.69,148.00,0.53,0.00,0.27,2.92,1.33,0.00,...,0.27,0.00,Essays,Involved persuasion,1.10,-4.23,2.52,3.96,5.16,6.62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
450,essays_text_97,126.00,4.92,65.00,0.00,0.00,0.79,2.38,0.79,0.00,...,0.00,0.00,Essays,Involved persuasion,2.17,-4.91,8.34,-6.43,1.02,-0.77
451,essays_text_98,163.00,4.36,76.00,0.00,0.00,0.00,1.84,0.61,0.00,...,0.61,0.00,Essays,Scientific exposition,-11.51,-4.75,4.16,-6.21,1.86,-3.50
452,essays_text_99,247.00,4.63,114.00,0.00,0.81,0.81,1.21,0.81,0.00,...,0.00,0.40,Essays,Involved persuasion,2.72,-3.02,-2.26,5.76,1.83,2.18
453,essays_text_9,315.00,4.66,147.00,0.00,0.00,1.59,1.27,0.00,0.00,...,0.00,0.00,Essays,Involved persuasion,1.75,-3.08,8.66,7.05,1.83,2.46


In [54]:
df4_new = pd.concat([df4_stats_stor, df4_stats_news, df4_stats_essays], axis=0)

In [55]:
df4_new

,Filename,Tokens,AWL,TTR,AMP,ANDC,CAUS,CC,CD,CONC,...,[WZPAST],[WZPRES],Genre,Closest Text Type,Dimension1,Dimension2,Dimension3,Dimension4,Dimension5,Dimension6
0,stories_text_0,1124.00,4.14,207.00,0.27,0.09,0.09,2.22,0.53,0.18,...,0.00,0.27,Stories,Imaginative narrative,-1.00,3.50,-2.12,-0.23,-0.03,-2.64
1,stories_text_10,5697.00,4.35,192.00,0.12,0.07,0.04,3.05,0.61,0.09,...,0.07,0.14,Stories,Learned exposition,-17.97,-5.86,4.86,-1.92,-1.19,-2.78
2,stories_text_11,9038.00,3.72,156.00,0.45,0.03,0.15,2.66,0.81,0.01,...,0.12,0.00,Stories,Involved persuasion,13.26,-2.72,-0.52,-1.87,-2.77,-1.74
3,stories_text_12,19671.00,4.45,242.00,0.23,0.12,0.08,2.06,0.98,0.09,...,0.05,0.07,Stories,Learned exposition,-14.15,-3.84,4.72,-0.96,-0.93,-1.18
4,stories_text_13,8300.00,4.02,168.00,0.57,0.13,0.17,2.88,0.48,0.06,...,0.07,0.00,Stories,Involved persuasion,3.51,0.31,-0.87,-0.18,-2.12,-0.98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
450,essays_text_97,126.00,4.92,65.00,0.00,0.00,0.79,2.38,0.79,0.00,...,0.00,0.00,Essays,Involved persuasion,2.17,-4.91,8.34,-6.43,1.02,-0.77
451,essays_text_98,163.00,4.36,76.00,0.00,0.00,0.00,1.84,0.61,0.00,...,0.61,0.00,Essays,Scientific exposition,-11.51,-4.75,4.16,-6.21,1.86,-3.50
452,essays_text_99,247.00,4.63,114.00,0.00,0.81,0.81,1.21,0.81,0.00,...,0.00,0.40,Essays,Involved persuasion,2.72,-3.02,-2.26,5.76,1.83,2.18
453,essays_text_9,315.00,4.66,147.00,0.00,0.00,1.59,1.27,0.00,0.00,...,0.00,0.00,Essays,Involved persuasion,1.75,-3.08,8.66,7.05,1.83,2.46


In [56]:
df4_new2 = df4_new

In [57]:
df4_new2

,Filename,Tokens,AWL,TTR,AMP,ANDC,CAUS,CC,CD,CONC,...,[WZPAST],[WZPRES],Genre,Closest Text Type,Dimension1,Dimension2,Dimension3,Dimension4,Dimension5,Dimension6
0,stories_text_0,1124.00,4.14,207.00,0.27,0.09,0.09,2.22,0.53,0.18,...,0.00,0.27,Stories,Imaginative narrative,-1.00,3.50,-2.12,-0.23,-0.03,-2.64
1,stories_text_10,5697.00,4.35,192.00,0.12,0.07,0.04,3.05,0.61,0.09,...,0.07,0.14,Stories,Learned exposition,-17.97,-5.86,4.86,-1.92,-1.19,-2.78
2,stories_text_11,9038.00,3.72,156.00,0.45,0.03,0.15,2.66,0.81,0.01,...,0.12,0.00,Stories,Involved persuasion,13.26,-2.72,-0.52,-1.87,-2.77,-1.74
3,stories_text_12,19671.00,4.45,242.00,0.23,0.12,0.08,2.06,0.98,0.09,...,0.05,0.07,Stories,Learned exposition,-14.15,-3.84,4.72,-0.96,-0.93,-1.18
4,stories_text_13,8300.00,4.02,168.00,0.57,0.13,0.17,2.88,0.48,0.06,...,0.07,0.00,Stories,Involved persuasion,3.51,0.31,-0.87,-0.18,-2.12,-0.98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
450,essays_text_97,126.00,4.92,65.00,0.00,0.00,0.79,2.38,0.79,0.00,...,0.00,0.00,Essays,Involved persuasion,2.17,-4.91,8.34,-6.43,1.02,-0.77
451,essays_text_98,163.00,4.36,76.00,0.00,0.00,0.00,1.84,0.61,0.00,...,0.61,0.00,Essays,Scientific exposition,-11.51,-4.75,4.16,-6.21,1.86,-3.50
452,essays_text_99,247.00,4.63,114.00,0.00,0.81,0.81,1.21,0.81,0.00,...,0.00,0.40,Essays,Involved persuasion,2.72,-3.02,-2.26,5.76,1.83,2.18
453,essays_text_9,315.00,4.66,147.00,0.00,0.00,1.59,1.27,0.00,0.00,...,0.00,0.00,Essays,Involved persuasion,1.75,-3.08,8.66,7.05,1.83,2.46


In [58]:
dummies = pd.get_dummies(df4_new2['Genre'])
dummies2 = pd.get_dummies(df4_new2['Closest Text Type'])
df4_new2 = pd.concat([df4_new2, dummies2, dummies], axis=1)

In [59]:
df4_new2

,Filename,Tokens,AWL,TTR,AMP,ANDC,CAUS,CC,CD,CONC,...,General narrative exposition,Imaginative narrative,Informational interaction,Involved persuasion,Learned exposition,Scientific exposition,Situated reportage,Essays,News,Stories
0,stories_text_0,1124.00,4.14,207.00,0.27,0.09,0.09,2.22,0.53,0.18,...,False,True,False,False,False,False,False,False,False,True
1,stories_text_10,5697.00,4.35,192.00,0.12,0.07,0.04,3.05,0.61,0.09,...,False,False,False,False,True,False,False,False,False,True
2,stories_text_11,9038.00,3.72,156.00,0.45,0.03,0.15,2.66,0.81,0.01,...,False,False,False,True,False,False,False,False,False,True
3,stories_text_12,19671.00,4.45,242.00,0.23,0.12,0.08,2.06,0.98,0.09,...,False,False,False,False,True,False,False,False,False,True
4,stories_text_13,8300.00,4.02,168.00,0.57,0.13,0.17,2.88,0.48,0.06,...,False,False,False,True,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
450,essays_text_97,126.00,4.92,65.00,0.00,0.00,0.79,2.38,0.79,0.00,...,False,False,False,True,False,False,False,True,False,False
451,essays_text_98,163.00,4.36,76.00,0.00,0.00,0.00,1.84,0.61,0.00,...,False,False,False,False,False,True,False,True,False,False
452,essays_text_99,247.00,4.63,114.00,0.00,0.81,0.81,1.21,0.81,0.00,...,False,False,False,True,False,False,False,True,False,False
453,essays_text_9,315.00,4.66,147.00,0.00,0.00,1.59,1.27,0.00,0.00,...,False,False,False,True,False,False,False,True,False,False


In [60]:
dummies = pd.get_dummies(df4_new2['Closest Text Type'])
df4_new2 = pd.concat([df4_new2, dummies], axis=1)

In [61]:
df4_new2 = df4_new2.replace(True,1).replace(False,0)

In [62]:
#df4_new2 = df4_new2.drop(['Genre', 'Closest Text Type'],axis=1)
df4_new2 = df4_new2.drop(['Genre'],axis=1)

In [63]:
df4_new2

,Filename,Tokens,AWL,TTR,AMP,ANDC,CAUS,CC,CD,CONC,...,Essays,News,Stories,General narrative exposition,Imaginative narrative,Informational interaction,Involved persuasion,Learned exposition,Scientific exposition,Situated reportage
0,stories_text_0,1124.00,4.14,207.00,0.27,0.09,0.09,2.22,0.53,0.18,...,0,0,1,0,1,0,0,0,0,0
1,stories_text_10,5697.00,4.35,192.00,0.12,0.07,0.04,3.05,0.61,0.09,...,0,0,1,0,0,0,0,1,0,0
2,stories_text_11,9038.00,3.72,156.00,0.45,0.03,0.15,2.66,0.81,0.01,...,0,0,1,0,0,0,1,0,0,0
3,stories_text_12,19671.00,4.45,242.00,0.23,0.12,0.08,2.06,0.98,0.09,...,0,0,1,0,0,0,0,1,0,0
4,stories_text_13,8300.00,4.02,168.00,0.57,0.13,0.17,2.88,0.48,0.06,...,0,0,1,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
450,essays_text_97,126.00,4.92,65.00,0.00,0.00,0.79,2.38,0.79,0.00,...,1,0,0,0,0,0,1,0,0,0
451,essays_text_98,163.00,4.36,76.00,0.00,0.00,0.00,1.84,0.61,0.00,...,1,0,0,0,0,0,0,0,1,0
452,essays_text_99,247.00,4.63,114.00,0.00,0.81,0.81,1.21,0.81,0.00,...,1,0,0,0,0,0,1,0,0,0
453,essays_text_9,315.00,4.66,147.00,0.00,0.00,1.59,1.27,0.00,0.00,...,1,0,0,0,0,0,1,0,0,0


In [64]:
df_y1 = df4_new2['General narrative exposition']
df_y2 = df4_new2['Imaginative narrative']
df_y3 = df4_new2['Informational interaction']
df_y4 = df4_new2['Involved persuasion']
df_y5 = df4_new2['Learned exposition']
df_y6 = df4_new2['Scientific exposition']
df_y7 = df4_new2['Situated reportage']

In [65]:
df_x = df4_new2.drop(['General narrative exposition', 
                      'Imaginative narrative',
                      'Informational interaction',
                      'Involved persuasion',
                      'Learned exposition',
                      'Scientific exposition',
                      'Situated reportage',
                      'Filename'], axis=1)

In [66]:
#df4_new2 = df4_new.drop(['Filename'],axis=1)

In [67]:
dfx = df4_new.copy()

In [68]:
dfx

,Filename,Tokens,AWL,TTR,AMP,ANDC,CAUS,CC,CD,CONC,...,[WZPAST],[WZPRES],Genre,Closest Text Type,Dimension1,Dimension2,Dimension3,Dimension4,Dimension5,Dimension6
0,stories_text_0,1124.00,4.14,207.00,0.27,0.09,0.09,2.22,0.53,0.18,...,0.00,0.27,Stories,Imaginative narrative,-1.00,3.50,-2.12,-0.23,-0.03,-2.64
1,stories_text_10,5697.00,4.35,192.00,0.12,0.07,0.04,3.05,0.61,0.09,...,0.07,0.14,Stories,Learned exposition,-17.97,-5.86,4.86,-1.92,-1.19,-2.78
2,stories_text_11,9038.00,3.72,156.00,0.45,0.03,0.15,2.66,0.81,0.01,...,0.12,0.00,Stories,Involved persuasion,13.26,-2.72,-0.52,-1.87,-2.77,-1.74
3,stories_text_12,19671.00,4.45,242.00,0.23,0.12,0.08,2.06,0.98,0.09,...,0.05,0.07,Stories,Learned exposition,-14.15,-3.84,4.72,-0.96,-0.93,-1.18
4,stories_text_13,8300.00,4.02,168.00,0.57,0.13,0.17,2.88,0.48,0.06,...,0.07,0.00,Stories,Involved persuasion,3.51,0.31,-0.87,-0.18,-2.12,-0.98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
450,essays_text_97,126.00,4.92,65.00,0.00,0.00,0.79,2.38,0.79,0.00,...,0.00,0.00,Essays,Involved persuasion,2.17,-4.91,8.34,-6.43,1.02,-0.77
451,essays_text_98,163.00,4.36,76.00,0.00,0.00,0.00,1.84,0.61,0.00,...,0.61,0.00,Essays,Scientific exposition,-11.51,-4.75,4.16,-6.21,1.86,-3.50
452,essays_text_99,247.00,4.63,114.00,0.00,0.81,0.81,1.21,0.81,0.00,...,0.00,0.40,Essays,Involved persuasion,2.72,-3.02,-2.26,5.76,1.83,2.18
453,essays_text_9,315.00,4.66,147.00,0.00,0.00,1.59,1.27,0.00,0.00,...,0.00,0.00,Essays,Involved persuasion,1.75,-3.08,8.66,7.05,1.83,2.46


In [69]:
closest_text = []
for d in dfx['Closest Text Type']:
    if d == 'General narrative exposition':
        closest_text.append(0)
    elif d == 'Imaginative narrative':
        closest_text.append(1)
    elif d == 'Informational interaction':
        closest_text.append(2)
    elif d == 'Involved persuasion':
        closest_text.append(3)
    elif d == 'Learned exposition':
        closest_text.append(4)
    elif d == 'Scientific exposition':
        closest_text.append(5)
    else:
        closest_text.append(6)

In [70]:
df_x['Closest Text Type'] = closest_text

In [71]:
df_x

,Tokens,AWL,TTR,AMP,ANDC,CAUS,CC,CD,CONC,COND,...,Closest Text Type,Dimension1,Dimension2,Dimension3,Dimension4,Dimension5,Dimension6,Essays,News,Stories
0,1124.00,4.14,207.00,0.27,0.09,0.09,2.22,0.53,0.18,0.36,...,1,-1.00,3.50,-2.12,-0.23,-0.03,-2.64,0,0,1
1,5697.00,4.35,192.00,0.12,0.07,0.04,3.05,0.61,0.09,0.14,...,4,-17.97,-5.86,4.86,-1.92,-1.19,-2.78,0,0,1
2,9038.00,3.72,156.00,0.45,0.03,0.15,2.66,0.81,0.01,0.13,...,3,13.26,-2.72,-0.52,-1.87,-2.77,-1.74,0,0,1
3,19671.00,4.45,242.00,0.23,0.12,0.08,2.06,0.98,0.09,0.20,...,4,-14.15,-3.84,4.72,-0.96,-0.93,-1.18,0,0,1
4,8300.00,4.02,168.00,0.57,0.13,0.17,2.88,0.48,0.06,0.25,...,3,3.51,0.31,-0.87,-0.18,-2.12,-0.98,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
450,126.00,4.92,65.00,0.00,0.00,0.79,2.38,0.79,0.00,0.00,...,3,2.17,-4.91,8.34,-6.43,1.02,-0.77,1,0,0
451,163.00,4.36,76.00,0.00,0.00,0.00,1.84,0.61,0.00,0.00,...,5,-11.51,-4.75,4.16,-6.21,1.86,-3.50,1,0,0
452,247.00,4.63,114.00,0.00,0.81,0.81,1.21,0.81,0.00,0.40,...,3,2.72,-3.02,-2.26,5.76,1.83,2.18,1,0,0
453,315.00,4.66,147.00,0.00,0.00,1.59,1.27,0.00,0.00,0.00,...,3,1.75,-3.08,8.66,7.05,1.83,2.46,1,0,0


In [72]:
df4_new['Closest Text Type'].unique()

array(['Imaginative narrative', 'Learned exposition',
       'Involved persuasion', 'General narrative exposition',
       'Situated reportage', 'Scientific exposition',
       'Informational interaction'], dtype=object)

In [73]:
df_dim_x = df_x[['Dimension1',
                 'Dimension2',
                 'Dimension3',
                 'Dimension4',
                 'Dimension5',
                 'Dimension6',
                 'Essays',
                 'News',
                 'Stories']]

In [74]:
df_dim_x

,Dimension1,Dimension2,Dimension3,Dimension4,Dimension5,Dimension6,Essays,News,Stories
0,-1.00,3.50,-2.12,-0.23,-0.03,-2.64,0,0,1
1,-17.97,-5.86,4.86,-1.92,-1.19,-2.78,0,0,1
2,13.26,-2.72,-0.52,-1.87,-2.77,-1.74,0,0,1
3,-14.15,-3.84,4.72,-0.96,-0.93,-1.18,0,0,1
4,3.51,0.31,-0.87,-0.18,-2.12,-0.98,0,0,1
...,...,...,...,...,...,...,...,...,...
450,2.17,-4.91,8.34,-6.43,1.02,-0.77,1,0,0
451,-11.51,-4.75,4.16,-6.21,1.86,-3.50,1,0,0
452,2.72,-3.02,-2.26,5.76,1.83,2.18,1,0,0
453,1.75,-3.08,8.66,7.05,1.83,2.46,1,0,0


In [75]:
#df_dim_x.corr()['Closest Text Type']

In [76]:
#df4_new2.to_csv('Text_types_dimensions.csv', index=False)

In [77]:
df_x.corr()['Closest Text Type']

Tokens       -0.071126
AWL           0.197040
TTR          -0.283214
AMP          -0.079636
ANDC         -0.077077
                ...   
Dimension5    0.336143
Dimension6   -0.062744
Essays        0.223243
News         -0.209490
Stories      -0.067024
Name: Closest Text Type, Length: 89, dtype: float64

In [78]:
df4_new3 = df_x.corr()['Closest Text Type'].reset_index()

In [79]:
df4_new3 = df4_new3[df4_new3['Closest Text Type'] < 1]

In [80]:
df_xx = df_x.copy()

In [81]:
df_xx = df_xx.drop([
                'Dimension1',
                'Dimension2',
                'Dimension3',
                'Dimension4',
                'Dimension5',
                'Dimension6'], axis=1)

In [82]:
df_xx

,Tokens,AWL,TTR,AMP,ANDC,CAUS,CC,CD,CONC,COND,...,[THATD],[WHCL],[WHOBJ],[WHSUB],[WZPAST],[WZPRES],Closest Text Type,Essays,News,Stories
0,1124.00,4.14,207.00,0.27,0.09,0.09,2.22,0.53,0.18,0.36,...,0.71,0.18,0.00,0.00,0.00,0.27,1,0,0,1
1,5697.00,4.35,192.00,0.12,0.07,0.04,3.05,0.61,0.09,0.14,...,0.05,0.02,0.02,0.19,0.07,0.14,4,0,0,1
2,9038.00,3.72,156.00,0.45,0.03,0.15,2.66,0.81,0.01,0.13,...,0.91,0.23,0.03,0.11,0.12,0.00,3,0,0,1
3,19671.00,4.45,242.00,0.23,0.12,0.08,2.06,0.98,0.09,0.20,...,0.24,0.07,0.06,0.44,0.05,0.07,4,0,0,1
4,8300.00,4.02,168.00,0.57,0.13,0.17,2.88,0.48,0.06,0.25,...,0.55,0.13,0.01,0.12,0.07,0.00,3,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
450,126.00,4.92,65.00,0.00,0.00,0.79,2.38,0.79,0.00,0.00,...,0.00,0.00,0.79,1.59,0.00,0.00,3,1,0,0
451,163.00,4.36,76.00,0.00,0.00,0.00,1.84,0.61,0.00,0.00,...,0.00,0.00,0.00,0.00,0.61,0.00,5,1,0,0
452,247.00,4.63,114.00,0.00,0.81,0.81,1.21,0.81,0.00,0.40,...,0.00,0.00,0.00,0.00,0.00,0.40,3,1,0,0
453,315.00,4.66,147.00,0.00,0.00,1.59,1.27,0.00,0.00,0.00,...,0.00,0.00,0.00,0.32,0.00,0.00,3,1,0,0


In [187]:
corr_head = df_xx.corr()['Closest Text Type'][df_xx.corr()['Closest Text Type'] != 1].sort_values(ascending=False).head()

In [188]:
corr_head

Essays    0.223243
CONJ      0.211919
AWL       0.197040
PHC       0.179598
OSUB      0.175387
Name: Closest Text Type, dtype: float64

In [196]:
from scipy import stats

print(stats.pearsonr(df_xx['Essays'], df_xx['Closest Text Type']))
print(stats.pearsonr(df_xx['CONJ'], df_xx['Closest Text Type']))
print(stats.pearsonr(df_xx['AWL'], df_xx['Closest Text Type']))
print(stats.pearsonr(df_xx['PHC'], df_xx['Closest Text Type']))
print(stats.pearsonr(df_xx['OSUB'], df_xx['Closest Text Type']))

PearsonRResult(statistic=0.22324258889333656, pvalue=1.5887575107253699e-07)
PearsonRResult(statistic=0.211919267169298, pvalue=6.705590104231095e-07)
PearsonRResult(statistic=0.19704042602552396, pvalue=3.959443479304768e-06)
PearsonRResult(statistic=0.17959826792239938, pvalue=2.6928619589584148e-05)
PearsonRResult(statistic=0.1753873847331515, pvalue=4.1674088198582255e-05)


In [84]:
#df_xx.corr()['Closest Text Type'].sort_values(ascending=False)[:6]

df_xx.corr()['Closest Text Type'][df_xx.corr()['Closest Text Type'] != 1].sort_values(ascending=True).head()

TTR       -0.283214
[PUBV]    -0.262575
[THATD]   -0.224993
News      -0.209490
TPP3      -0.208917
Name: Closest Text Type, dtype: float64

In [197]:
print(stats.pearsonr(df_xx['TTR'], df_xx['Closest Text Type']))
print(stats.pearsonr(df_xx['[PUBV]'], df_xx['Closest Text Type']))
print(stats.pearsonr(df_xx['[THATD]'], df_xx['Closest Text Type']))
print(stats.pearsonr(df_xx['News'], df_xx['Closest Text Type']))
print(stats.pearsonr(df_xx['TPP3'], df_xx['Closest Text Type']))

PearsonRResult(statistic=-0.2832140980969962, pvalue=2.030860986029067e-11)
PearsonRResult(statistic=-0.2625752598492189, pvalue=5.769088585335677e-10)
PearsonRResult(statistic=-0.22499273844324408, pvalue=1.2629537013460042e-07)
PearsonRResult(statistic=-0.20948998961071402, pvalue=9.04157923893375e-07)
PearsonRResult(statistic=-0.2089166378020027, pvalue=9.69746189577292e-07)


In [85]:
df4_new2_stories = df_xx[df_xx['Stories'] == 1]
df4_new2_news = df_xx[df_xx['News'] == 1]
df4_new2_essays = df_xx[df_xx['Essays'] == 1]

In [86]:
df4_new2_stories

,Tokens,AWL,TTR,AMP,ANDC,CAUS,CC,CD,CONC,COND,...,[THATD],[WHCL],[WHOBJ],[WHSUB],[WZPAST],[WZPRES],Closest Text Type,Essays,News,Stories
0,1124.0,4.14,207.0,0.27,0.09,0.09,2.22,0.53,0.18,0.36,...,0.71,0.18,0.00,0.00,0.00,0.27,1,0,0,1
1,5697.0,4.35,192.0,0.12,0.07,0.04,3.05,0.61,0.09,0.14,...,0.05,0.02,0.02,0.19,0.07,0.14,4,0,0,1
2,9038.0,3.72,156.0,0.45,0.03,0.15,2.66,0.81,0.01,0.13,...,0.91,0.23,0.03,0.11,0.12,0.00,3,0,0,1
3,19671.0,4.45,242.0,0.23,0.12,0.08,2.06,0.98,0.09,0.20,...,0.24,0.07,0.06,0.44,0.05,0.07,4,0,0,1
4,8300.0,4.02,168.0,0.57,0.13,0.17,2.88,0.48,0.06,0.25,...,0.55,0.13,0.01,0.12,0.07,0.00,3,0,0,1
5,12573.0,3.88,191.0,0.05,0.01,0.02,1.48,0.68,0.02,0.18,...,0.56,0.16,0.00,0.04,0.10,0.04,0,0,0,1
6,29520.0,4.47,194.0,0.47,0.18,0.14,3.00,0.88,0.13,0.40,...,0.19,0.14,0.19,0.48,0.04,0.02,3,0,0,1
7,23.0,3.57,13.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,6,0,0,1
8,157.0,4.00,96.0,0.00,0.00,0.00,5.10,2.55,0.00,1.91,...,0.00,0.00,0.00,0.00,0.00,0.00,3,0,0,1
9,1639.0,4.05,216.0,0.12,0.00,0.00,2.20,0.67,0.00,0.49,...,0.85,0.06,0.00,0.18,0.00,0.00,0,0,0,1


In [87]:
df1_train = df4_new2_stories[:8]
df2_train = df4_new2_news[:34]
df3_train = df4_new2_essays[:227]

df1_test = df4_new2_stories[8:]
df2_test = df4_new2_news[34:]
df3_test = df4_new2_essays[227:]

In [88]:
df1_train

,Tokens,AWL,TTR,AMP,ANDC,CAUS,CC,CD,CONC,COND,...,[THATD],[WHCL],[WHOBJ],[WHSUB],[WZPAST],[WZPRES],Closest Text Type,Essays,News,Stories
0,1124.0,4.14,207.0,0.27,0.09,0.09,2.22,0.53,0.18,0.36,...,0.71,0.18,0.00,0.00,0.00,0.27,1,0,0,1
1,5697.0,4.35,192.0,0.12,0.07,0.04,3.05,0.61,0.09,0.14,...,0.05,0.02,0.02,0.19,0.07,0.14,4,0,0,1
2,9038.0,3.72,156.0,0.45,0.03,0.15,2.66,0.81,0.01,0.13,...,0.91,0.23,0.03,0.11,0.12,0.00,3,0,0,1
3,19671.0,4.45,242.0,0.23,0.12,0.08,2.06,0.98,0.09,0.20,...,0.24,0.07,0.06,0.44,0.05,0.07,4,0,0,1
4,8300.0,4.02,168.0,0.57,0.13,0.17,2.88,0.48,0.06,0.25,...,0.55,0.13,0.01,0.12,0.07,0.00,3,0,0,1
5,12573.0,3.88,191.0,0.05,0.01,0.02,1.48,0.68,0.02,0.18,...,0.56,0.16,0.00,0.04,0.10,0.04,0,0,0,1
6,29520.0,4.47,194.0,0.47,0.18,0.14,3.00,0.88,0.13,0.40,...,0.19,0.14,0.19,0.48,0.04,0.02,3,0,0,1
7,23.0,3.57,13.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,6,0,0,1


In [89]:
y_train1 = df1_train['Closest Text Type']
x_train1 = df1_train.drop(['Closest Text Type'], axis = 1)

In [90]:
x_train1

,Tokens,AWL,TTR,AMP,ANDC,CAUS,CC,CD,CONC,COND,...,[SUAV],[THATD],[WHCL],[WHOBJ],[WHSUB],[WZPAST],[WZPRES],Essays,News,Stories
0,1124.0,4.14,207.0,0.27,0.09,0.09,2.22,0.53,0.18,0.36,...,0.36,0.71,0.18,0.00,0.00,0.00,0.27,0,0,1
1,5697.0,4.35,192.0,0.12,0.07,0.04,3.05,0.61,0.09,0.14,...,0.18,0.05,0.02,0.02,0.19,0.07,0.14,0,0,1
2,9038.0,3.72,156.0,0.45,0.03,0.15,2.66,0.81,0.01,0.13,...,0.41,0.91,0.23,0.03,0.11,0.12,0.00,0,0,1
3,19671.0,4.45,242.0,0.23,0.12,0.08,2.06,0.98,0.09,0.20,...,0.23,0.24,0.07,0.06,0.44,0.05,0.07,0,0,1
4,8300.0,4.02,168.0,0.57,0.13,0.17,2.88,0.48,0.06,0.25,...,0.58,0.55,0.13,0.01,0.12,0.07,0.00,0,0,1
5,12573.0,3.88,191.0,0.05,0.01,0.02,1.48,0.68,0.02,0.18,...,0.30,0.56,0.16,0.00,0.04,0.10,0.04,0,0,1
6,29520.0,4.47,194.0,0.47,0.18,0.14,3.00,0.88,0.13,0.40,...,0.37,0.19,0.14,0.19,0.48,0.04,0.02,0,0,1
7,23.0,3.57,13.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,1


In [91]:
y_train2 = df2_train['Closest Text Type']
x_train2 = df2_train.drop(['Closest Text Type'], axis = 1)

In [92]:
y_train3 = df3_train['Closest Text Type']
x_train3 = df3_train.drop(['Closest Text Type'], axis = 1)

In [93]:
y_test1 = df1_test['Closest Text Type']
x_test1 = df1_test.drop(['Closest Text Type'], axis = 1)

In [94]:
y_test2 = df2_test['Closest Text Type']
x_test2 = df2_test.drop(['Closest Text Type'], axis = 1)

In [95]:
y_test3 = df3_test['Closest Text Type']
x_test3 = df3_test.drop(['Closest Text Type'], axis = 1)

In [96]:
y_test3

227    3
228    3
229    3
230    5
231    3
      ..
450    3
451    5
452    3
453    3
454    3
Name: Closest Text Type, Length: 228, dtype: int64

In [97]:
X_train = pd.concat([x_train1, x_train2, x_train3], axis = 0)

In [98]:
y_train = pd.concat([y_train1, y_train2, y_train3], axis = 0)

In [99]:
X_test = pd.concat([x_test1, x_test2, x_test3], axis = 0)

In [100]:
y_test = pd.concat([y_test1, y_test2, y_test3], axis = 0)

In [101]:
#X_train

In [102]:
X_test

,Tokens,AWL,TTR,AMP,ANDC,CAUS,CC,CD,CONC,COND,...,[SUAV],[THATD],[WHCL],[WHOBJ],[WHSUB],[WZPAST],[WZPRES],Essays,News,Stories
8,157.00,4.00,96.00,0.00,0.00,0.00,5.10,2.55,0.00,1.91,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,1
9,1639.00,4.05,216.00,0.12,0.00,0.00,2.20,0.67,0.00,0.49,...,0.61,0.85,0.06,0.00,0.18,0.00,0.00,0,0,1
10,2333.00,3.85,200.00,0.21,0.09,0.09,2.96,0.43,0.13,0.47,...,0.51,1.63,0.17,0.00,0.13,0.00,0.00,0,0,1
11,1281.00,3.92,192.00,0.08,0.08,0.31,2.11,1.09,0.16,0.55,...,0.31,0.70,0.23,0.08,0.00,0.16,0.00,0,0,1
12,1865.00,3.94,201.00,0.16,0.05,0.11,2.25,0.80,0.16,0.27,...,0.32,0.54,0.21,0.05,0.11,0.05,0.05,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
450,126.00,4.92,65.00,0.00,0.00,0.79,2.38,0.79,0.00,0.00,...,0.00,0.00,0.00,0.79,1.59,0.00,0.00,1,0,0
451,163.00,4.36,76.00,0.00,0.00,0.00,1.84,0.61,0.00,0.00,...,0.61,0.00,0.00,0.00,0.00,0.61,0.00,1,0,0
452,247.00,4.63,114.00,0.00,0.81,0.81,1.21,0.81,0.00,0.40,...,0.00,0.00,0.00,0.00,0.00,0.00,0.40,1,0,0
453,315.00,4.66,147.00,0.00,0.00,1.59,1.27,0.00,0.00,0.00,...,0.95,0.00,0.00,0.00,0.32,0.00,0.00,1,0,0


In [103]:
#print(df4_new3.sort_values(by='Closest Text Type', ascending=True).head())

In [104]:
closest_parms = pd.concat([df_xx.corr()['Closest Text Type'][df_xx.corr()['Closest Text Type'] != 1].sort_values(ascending=False).head(), df_xx.corr()['Closest Text Type'][df_xx.corr()['Closest Text Type'] != 1].sort_values(ascending=True).head()])
#closest_parms

feature = closest_parms.index
score = closest_parms.values
correlation = pd.DataFrame()
correlation['Параметр'] = feature
correlation['Обозначение'] = ['Is the document essay?',
                              'Conjuncts',
                              'Average word length',
                              'The «and» conjunctive',
                              'Phrasal coordination',
                              'Other adverbial subordinators',
                              'Public verbs',
                              'Subordinator "that" deletion',
                              'Is the document news?',
                              'Third person pronouns']
correlation['Корреляция'] = score

correlation

,Параметр,Обозначение,Корреляция
0,Essays,Is the document essay?,0.223243
1,CONJ,Conjuncts,0.211919
2,AWL,Average word length,0.197040
3,PHC,The «and» conjunctive,0.179598
4,OSUB,Phrasal coordination,0.175387
5,TTR,Other adverbial subordinators,-0.283214
6,[PUBV],Public verbs,-0.262575
7,[THATD],"Subordinator ""that"" deletion",-0.224993
8,News,Is the document news?,-0.209490
9,TPP3,Third person pronouns,-0.208917


In [105]:
#closest_parms.index

In [106]:
X_train = X_train[['Essays', 'CONJ', 'AWL', 'PHC', 'OSUB', 'TTR', '[PUBV]', '[THATD]',
           'News', 'TPP3', 'Stories']]

In [107]:
X_test = X_test[['Essays', 'CONJ', 'AWL', 'PHC', 'OSUB', 'TTR', '[PUBV]', '[THATD]',
           'News', 'TPP3', 'Stories']]

In [108]:
y_train

0      1
1      4
2      3
3      4
4      3
      ..
222    4
223    3
224    0
225    3
226    4
Name: Closest Text Type, Length: 269, dtype: int64

In [110]:
X_train

,Essays,CONJ,AWL,PHC,OSUB,TTR,[PUBV],[THATD],News,TPP3,Stories
0,0,0.36,4.14,1.07,0.18,207.0,1.25,0.71,0,5.25,1
1,0,0.19,4.35,1.25,0.11,192.0,0.26,0.05,0,0.53,1
2,0,0.02,3.72,0.64,0.03,156.0,1.19,0.91,0,2.20,1
3,0,0.17,4.45,0.99,0.18,242.0,0.54,0.24,0,2.75,1
4,0,0.01,4.02,0.78,0.14,168.0,2.01,0.55,0,6.82,1
...,...,...,...,...,...,...,...,...,...,...,...
222,1,0.00,4.42,1.15,0.00,69.0,0.57,0.00,0,0.00,0
223,1,0.70,4.44,0.70,0.00,106.0,0.00,0.35,0,3.14,0
224,1,0.57,4.39,0.00,0.00,85.0,0.00,0.57,0,1.14,0
225,1,0.47,4.39,0.95,0.47,103.0,0.95,0.95,0,1.90,0


In [111]:
closest_parms

Essays     0.223243
CONJ       0.211919
AWL        0.197040
PHC        0.179598
OSUB       0.175387
TTR       -0.283214
[PUBV]    -0.262575
[THATD]   -0.224993
News      -0.209490
TPP3      -0.208917
Name: Closest Text Type, dtype: float64

In [112]:
parms = []
for c in closest_parms.index:
    parms.append(c)
parms.append('Essays')
parms.append('Stories')
parms.append('Closest Text Type')

In [113]:
parms = ['CONJ',
         'AWL',
         'PHC',
         'OSUB',
         'TTR',
         '[PUBV]',
         '[THATD]',
         'TPP3',
         'News',
         'Essays',
         'Stories',
         'Closest Text Type']

In [115]:
df_xx = df_xx[parms]

In [116]:
df_xx

,CONJ,AWL,PHC,OSUB,TTR,[PUBV],[THATD],TPP3,News,Essays,Stories,Closest Text Type
0,0.36,4.14,1.07,0.18,207.00,1.25,0.71,5.25,0,0,1,1
1,0.19,4.35,1.25,0.11,192.00,0.26,0.05,0.53,0,0,1,4
2,0.02,3.72,0.64,0.03,156.00,1.19,0.91,2.20,0,0,1,3
3,0.17,4.45,0.99,0.18,242.00,0.54,0.24,2.75,0,0,1,4
4,0.01,4.02,0.78,0.14,168.00,2.01,0.55,6.82,0,0,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...
450,0.79,4.92,0.00,0.00,65.00,0.00,0.00,5.56,0,1,0,3
451,0.61,4.36,1.23,0.00,76.00,0.00,0.00,0.61,0,1,0,5
452,1.62,4.63,0.81,0.00,114.00,0.81,0.00,0.81,0,1,0,3
453,2.22,4.66,1.59,0.00,147.00,0.63,0.00,2.54,0,1,0,3


In [117]:
df_y0 = df_xx['Closest Text Type']

In [118]:
df_x = df_xx.drop(['Closest Text Type'], axis=1)

In [119]:
df_x_new = df_x.values

In [120]:
df_x

,CONJ,AWL,PHC,OSUB,TTR,[PUBV],[THATD],TPP3,News,Essays,Stories
0,0.36,4.14,1.07,0.18,207.00,1.25,0.71,5.25,0,0,1
1,0.19,4.35,1.25,0.11,192.00,0.26,0.05,0.53,0,0,1
2,0.02,3.72,0.64,0.03,156.00,1.19,0.91,2.20,0,0,1
3,0.17,4.45,0.99,0.18,242.00,0.54,0.24,2.75,0,0,1
4,0.01,4.02,0.78,0.14,168.00,2.01,0.55,6.82,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
450,0.79,4.92,0.00,0.00,65.00,0.00,0.00,5.56,0,1,0
451,0.61,4.36,1.23,0.00,76.00,0.00,0.00,0.61,0,1,0
452,1.62,4.63,0.81,0.00,114.00,0.81,0.00,0.81,0,1,0
453,2.22,4.66,1.59,0.00,147.00,0.63,0.00,2.54,0,1,0


In [121]:
X = df_x_new

In [168]:
df_x

,CONJ,AWL,PHC,OSUB,TTR,[PUBV],[THATD],TPP3,News,Essays,Stories
0,0.36,4.14,1.07,0.18,207.00,1.25,0.71,5.25,0,0,1
1,0.19,4.35,1.25,0.11,192.00,0.26,0.05,0.53,0,0,1
2,0.02,3.72,0.64,0.03,156.00,1.19,0.91,2.20,0,0,1
3,0.17,4.45,0.99,0.18,242.00,0.54,0.24,2.75,0,0,1
4,0.01,4.02,0.78,0.14,168.00,2.01,0.55,6.82,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
450,0.79,4.92,0.00,0.00,65.00,0.00,0.00,5.56,0,1,0
451,0.61,4.36,1.23,0.00,76.00,0.00,0.00,0.61,0,1,0
452,1.62,4.63,0.81,0.00,114.00,0.81,0.00,0.81,0,1,0
453,2.22,4.66,1.59,0.00,147.00,0.63,0.00,2.54,0,1,0


### Подготовка тренировочных и тестовых данных (Dimensions)

In [123]:
df_dim_xx = df_dim_x.copy()
df_dim_xx['Closest Text Type'] = df_y0

df_dim_essays = df_dim_xx[df_dim_xx['Essays'] == 1]
df_dim_news = df_dim_xx[df_dim_xx['News'] == 1]
df_dim_stories = df_dim_xx[df_dim_xx['Stories'] == 1]


df_dim_essays = df_dim_essays.drop(['Essays','News','Stories'], axis=1)
df_dim_news = df_dim_news.drop(['Essays','News','Stories'], axis=1)
df_dim_stories = df_dim_stories.drop(['Essays','News','Stories'], axis=1)

X_essays_train = df_dim_essays[:len(df_dim_essays)//2]
X_news_train = df_dim_news[:len(df_dim_news)//2]
X_stories_train = df_dim_stories[:len(df_dim_stories)//2]

X_essays_test = df_dim_essays[len(df_dim_essays)//2:]
X_news_test = df_dim_news[len(df_dim_news)//2:]
X_stories_test = df_dim_stories[len(df_dim_stories)//2:]

X_train_dims = pd.concat([X_essays_train, X_news_train, X_stories_train], axis=0)
X_test_dims = pd.concat([X_essays_test, X_news_test, X_stories_test], axis=0)

y_train_dims = X_train_dims['Closest Text Type']
X_train_dims = X_train_dims.drop(['Closest Text Type'], axis=1)

y_test_dims = X_test_dims['Closest Text Type']
X_test_dims = X_test_dims.drop(['Closest Text Type'], axis=1)

df_dim_xx.corr()['Closest Text Type'][df_dim_xx.corr()['Closest Text Type'] != 1].sort_values(ascending=False)

Dimension5    0.336143
Essays        0.223243
Dimension3    0.154404
Dimension6   -0.062744
Stories      -0.067024
Dimension1   -0.153767
Dimension4   -0.164398
News         -0.209490
Dimension2   -0.316524
Name: Closest Text Type, dtype: float64

In [200]:
print(stats.pearsonr(df_dim_xx['Dimension5'], df_dim_xx['Closest Text Type']))
print(stats.pearsonr(df_dim_xx['Dimension3'], df_dim_xx['Closest Text Type']))
print(stats.pearsonr(df_dim_xx['Dimension6'], df_dim_xx['Closest Text Type']))
print(stats.pearsonr(df_dim_xx['Dimension1'], df_dim_xx['Closest Text Type']))
print(stats.pearsonr(df_dim_xx['Dimension4'], df_dim_xx['Closest Text Type']))
print(stats.pearsonr(df_dim_xx['Dimension2'], df_dim_xx['Closest Text Type']))

PearsonRResult(statistic=0.3361427050935328, pvalue=9.913836552675914e-16)
PearsonRResult(statistic=0.15440439648998774, pvalue=0.00031658293048738407)
PearsonRResult(statistic=-0.06274442599739727, pvalue=0.14536360077296848)
PearsonRResult(statistic=-0.15376668167558594, pvalue=0.000335422741375932)
PearsonRResult(statistic=-0.1643976330727352, pvalue=0.00012427121372380367)
PearsonRResult(statistic=-0.31652420501167, pvalue=4.964536355228386e-14)


In [124]:
df_dim_xx_train = df_dim_xx.drop(['Closest Text Type'], axis=1)

In [125]:
df_dim_xx.to_csv('news_essays_stories_dimensions.csv', index=False)

### Сопоставление алгоритмов разбиения выборки на обучающую и тестовую:

Будет рассмотрено два возможных варианта распределения, с последующим сравнением в плане точности предсказания значений.

1. Разбить датасет на составляющие по жанрам (эссе/новости/рассказы), отобрать по половинке из каждого и собрать воедино в обучающую и тестовую выборки.
2. Использовать методику train_test_split() и разбить автоматически 50 на 50.

### Нейросеть (без Dimensions)

In [126]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
X_scale = min_max_scaler.fit_transform(X)

#### Разметка по train_test_split

In [127]:
(trainX, testX, trainY, testY) = train_test_split(X_scale,
    df_y0, test_size=0.5, random_state=42)

In [128]:
from sklearn.preprocessing import LabelBinarizer

lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)

In [129]:
trainY.shape

(270, 6)

In [130]:
trainX.shape

(270, 11)

In [131]:
trainY

array([[0, 0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 1],
       [0, 0, 1, 0, 0, 0]])

In [132]:
from keras.models import Sequential
model = Sequential()
model.add(Dense(1024, input_shape=(11,), activation="sigmoid"))
model.add(Dense(512, activation="sigmoid"))
model.add(Dense(len(lb.classes_), activation="softmax"))

In [133]:
model.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [134]:
H = model.fit(trainX, trainY, validation_data=(testX, testY),
    epochs=100, batch_size=32) 

Epoch 1/100
9/9 [==============================] - 1s 25ms/step - loss: 1.6691 - accuracy: 0.3519 - val_loss: 1.6025 - val_accuracy: 0.4111
Epoch 2/100
9/9 [==============================] - 0s 10ms/step - loss: 1.7495 - accuracy: 0.4037 - val_loss: 1.5141 - val_accuracy: 0.4111
Epoch 3/100
9/9 [==============================] - 0s 13ms/step - loss: 1.5286 - accuracy: 0.4519 - val_loss: 1.5132 - val_accuracy: 0.4111
Epoch 4/100
9/9 [==============================] - 0s 11ms/step - loss: 1.5045 - accuracy: 0.4519 - val_loss: 1.4056 - val_accuracy: 0.4111
Epoch 5/100
9/9 [==============================] - 0s 10ms/step - loss: 1.5022 - accuracy: 0.4519 - val_loss: 1.4327 - val_accuracy: 0.4111
Epoch 6/100
9/9 [==============================] - 0s 10ms/step - loss: 1.4738 - accuracy: 0.4519 - val_loss: 1.4629 - val_accuracy: 0.4111
Epoch 7/100
9/9 [==============================] - 0s 10ms/step - loss: 1.4429 - accuracy: 0.4519 - val_loss: 1.3910 - val_accuracy: 0.4111
Epoch 8/100
9/9 [===

In [135]:
predictions = model.predict(testX, batch_size=32)
predictions2 = predictions.argmax(axis=1)

9/9 [==============================] - 0s 2ms/step


In [136]:
predictions2

array([3, 3, 3, 3, 2, 0, 2, 2, 3, 4, 0, 3, 3, 3, 3, 3, 2, 0, 2, 3, 2, 2,
       2, 2, 3, 2, 2, 0, 2, 3, 2, 2, 2, 2, 0, 2, 3, 3, 2, 3, 2, 3, 0, 3,
       2, 3, 3, 2, 2, 0, 3, 2, 3, 2, 2, 2, 2, 3, 0, 2, 2, 2, 0, 2, 3, 3,
       2, 2, 2, 2, 0, 2, 3, 2, 3, 3, 3, 2, 3, 0, 3, 3, 3, 3, 2, 2, 3, 0,
       2, 3, 2, 3, 3, 3, 2, 3, 3, 0, 2, 3, 2, 3, 3, 2, 3, 3, 2, 2, 2, 3,
       0, 3, 3, 2, 3, 3, 2, 3, 3, 2, 3, 2, 2, 3, 2, 0, 3, 0, 3, 2, 3, 3,
       3, 3, 3, 3, 4, 3, 2, 3, 3, 0, 2, 3, 0, 3, 2, 0, 2, 3, 2, 3, 3, 3,
       2, 0, 2, 3, 2, 3, 3, 0, 2, 3, 2, 3, 2, 3, 3, 2, 3, 3, 2, 2, 3, 3,
       2, 2, 3, 2, 2, 0, 2, 2, 3, 3, 2, 0, 3, 2, 2, 3, 0, 3, 3, 3, 3, 2,
       3, 2, 2, 3, 3, 3, 2, 3, 3, 3, 3, 0, 2, 0, 0, 3, 2, 3, 0, 3, 3, 3,
       0, 0, 0, 2, 2, 2, 3, 2, 3, 3, 3, 2, 2, 3, 3, 3, 2, 2, 3, 3, 2, 3,
       2, 2, 0, 3, 3, 2, 0, 2, 2, 3, 2, 3, 3, 2, 2, 3, 3, 2, 3, 2, 3, 2,
       0, 3, 3, 3, 2, 0], dtype=int64)

In [137]:
from sklearn.metrics import accuracy_score
testY_arg = testY.argmax(axis=1)
accuracy_score(testY_arg, predictions2)

0.5370370370370371

#### Половинчатая разметка всего

In [138]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
X_s_train = min_max_scaler.fit_transform(X_train)
X_s_test = min_max_scaler.fit_transform(X_test)

In [139]:
lb = LabelBinarizer()
y_s_train = lb.fit_transform(y_train)
y_s_test = lb.transform(y_test)

In [140]:
from keras.models import Sequential
model = Sequential()
model.add(Dense(1024, input_shape=(11,), activation="sigmoid"))
model.add(Dense(512, activation="sigmoid"))
model.add(Dense(len(lb.classes_), activation="softmax"))

model.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

H = model.fit(X_s_train, y_s_train, validation_data=(X_s_test, y_s_test),
    epochs=100, batch_size=32) 

Epoch 1/100
9/9 [==============================] - 1s 22ms/step - loss: 1.8640 - accuracy: 0.2454 - val_loss: 1.5814 - val_accuracy: 0.4428
Epoch 2/100
9/9 [==============================] - 0s 11ms/step - loss: 1.6306 - accuracy: 0.4201 - val_loss: 1.4986 - val_accuracy: 0.4428
Epoch 3/100
9/9 [==============================] - 0s 10ms/step - loss: 1.5226 - accuracy: 0.4201 - val_loss: 1.5206 - val_accuracy: 0.4428
Epoch 4/100
9/9 [==============================] - 0s 10ms/step - loss: 1.5466 - accuracy: 0.3383 - val_loss: 1.4623 - val_accuracy: 0.4428
Epoch 5/100
9/9 [==============================] - 0s 9ms/step - loss: 1.5167 - accuracy: 0.3532 - val_loss: 1.4464 - val_accuracy: 0.4428
Epoch 6/100
9/9 [==============================] - 0s 10ms/step - loss: 1.5505 - accuracy: 0.4201 - val_loss: 1.4765 - val_accuracy: 0.4945
Epoch 7/100
9/9 [==============================] - 0s 10ms/step - loss: 1.4825 - accuracy: 0.4201 - val_loss: 1.4320 - val_accuracy: 0.4428
Epoch 8/100
9/9 [====

In [141]:
predictions = model.predict(X_s_test, batch_size=32)
predictions2 = predictions.argmax(axis=1)

9/9 [==============================] - 0s 1ms/step


In [142]:
from sklearn.metrics import accuracy_score
testY_arg = y_s_test.argmax(axis=1)
accuracy_score(testY_arg, predictions2)

0.5719557195571956

# Пайплайн

### Пайплайн (без Dimensions)

In [143]:
df_xx.to_csv('news_essays_stories_features.csv', index=False)

In [144]:
df_x

,CONJ,AWL,PHC,OSUB,TTR,[PUBV],[THATD],TPP3,News,Essays,Stories
0,0.36,4.14,1.07,0.18,207.00,1.25,0.71,5.25,0,0,1
1,0.19,4.35,1.25,0.11,192.00,0.26,0.05,0.53,0,0,1
2,0.02,3.72,0.64,0.03,156.00,1.19,0.91,2.20,0,0,1
3,0.17,4.45,0.99,0.18,242.00,0.54,0.24,2.75,0,0,1
4,0.01,4.02,0.78,0.14,168.00,2.01,0.55,6.82,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
450,0.79,4.92,0.00,0.00,65.00,0.00,0.00,5.56,0,1,0
451,0.61,4.36,1.23,0.00,76.00,0.00,0.00,0.61,0,1,0
452,1.62,4.63,0.81,0.00,114.00,0.81,0.00,0.81,0,1,0
453,2.22,4.66,1.59,0.00,147.00,0.63,0.00,2.54,0,1,0


In [145]:
X_train

,Essays,CONJ,AWL,PHC,OSUB,TTR,[PUBV],[THATD],News,TPP3,Stories
0,0,0.36,4.14,1.07,0.18,207.0,1.25,0.71,0,5.25,1
1,0,0.19,4.35,1.25,0.11,192.0,0.26,0.05,0,0.53,1
2,0,0.02,3.72,0.64,0.03,156.0,1.19,0.91,0,2.20,1
3,0,0.17,4.45,0.99,0.18,242.0,0.54,0.24,0,2.75,1
4,0,0.01,4.02,0.78,0.14,168.0,2.01,0.55,0,6.82,1
...,...,...,...,...,...,...,...,...,...,...,...
222,1,0.00,4.42,1.15,0.00,69.0,0.57,0.00,0,0.00,0
223,1,0.70,4.44,0.70,0.00,106.0,0.00,0.35,0,3.14,0
224,1,0.57,4.39,0.00,0.00,85.0,0.00,0.57,0,1.14,0
225,1,0.47,4.39,0.95,0.47,103.0,0.95,0.95,0,1.90,0


#### Разметка по соотношению 50:50

In [182]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

trainX, testX, trainY, testY = train_test_split(df_x, df_y0,
                                                    random_state=0, test_size = 0.5)
estimators = [('scaler', StandardScaler()), 
                 ('classifier', RandomForestClassifier(random_state = 42, n_estimators = 1500)),
             ]

pipe = Pipeline(estimators)

pipe.fit(trainX, trainY).score(testX, testY)

0.6370370370370371

#### Половинчатая разметка "половинчатых" датасетов

In [181]:
estimators = [('scaler', StandardScaler()), 
                 ('classifier', RandomForestClassifier(random_state = 42, n_estimators = 1500)),
             ]

pipe = Pipeline(estimators)

pipe.fit(X_train, y_train).score(X_test, y_test)

0.6199261992619927

#### Отсюда можно сделать вывод, что разбивать общую выборку через *train_test_split* гораздо выгоднее, чем отбирать по половинке из каждой категории текстов и собирать их в обучающую и тестовую порции.

### Пайплайн (Dimensions)

#### Разметка по соотношению 50:50 со сплитом

In [148]:
trainX, testX, trainY, testY = train_test_split(df_dim_x, df_y0,
                                                    random_state=0, test_size = 0.5)
estimators = [('scaler', StandardScaler()), 
                 ('classifier', RandomForestClassifier(random_state = 42, n_estimators = 1500)),
             ]

pipe = Pipeline(estimators)

pipe.fit(trainX, trainY).score(testX, testY)

0.8407407407407408

#### Половинчатая разметка "половинчатых" датасетов

In [149]:
estimators = [('scaler', StandardScaler()), 
                 ('classifier', RandomForestClassifier(random_state = 42, n_estimators = 1500)),
             ]

pipe = Pipeline(estimators)

pipe.fit(X_train_dims, y_train_dims).score(X_test_dims, y_test_dims)

0.8597785977859779

# Нейросеть (Dimensions)

### Анализ нейросетью через Dimensions: автоматическое деление выборки 50 на 50 через *train_test_split*

In [150]:
df_dim_x2 = df_dim_x.drop(['Essays','News','Stories'], axis=1)

In [151]:
from sklearn.preprocessing import LabelBinarizer

min_max_scaler = preprocessing.MinMaxScaler()
X_scale = min_max_scaler.fit_transform(df_dim_x2)
X_scale

(trainX, testX, trainY, testY) = train_test_split(X_scale,
    df_y0, test_size=0.5, random_state=42)

lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)

In [152]:
from keras.models import Sequential
model = Sequential()
model.add(Dense(1024, input_shape=(6,), activation="sigmoid"))
model.add(Dense(512, activation="sigmoid"))
model.add(Dense(len(lb.classes_), activation="softmax"))

model.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

H = model.fit(trainX, trainY, validation_data=(testX, testY),
    epochs=100, batch_size=32) 

Epoch 1/100
9/9 [==============================] - 1s 22ms/step - loss: 1.7552 - accuracy: 0.3111 - val_loss: 1.7185 - val_accuracy: 0.4111
Epoch 2/100
9/9 [==============================] - 0s 10ms/step - loss: 1.7163 - accuracy: 0.2889 - val_loss: 1.6682 - val_accuracy: 0.4111
Epoch 3/100
9/9 [==============================] - 0s 10ms/step - loss: 1.5766 - accuracy: 0.4519 - val_loss: 1.4719 - val_accuracy: 0.4111
Epoch 4/100
9/9 [==============================] - 0s 9ms/step - loss: 1.4897 - accuracy: 0.4519 - val_loss: 1.4826 - val_accuracy: 0.4111
Epoch 5/100
9/9 [==============================] - 0s 9ms/step - loss: 1.4798 - accuracy: 0.4519 - val_loss: 1.4416 - val_accuracy: 0.4111
Epoch 6/100
9/9 [==============================] - 0s 9ms/step - loss: 1.4353 - accuracy: 0.4556 - val_loss: 1.4304 - val_accuracy: 0.4111
Epoch 7/100
9/9 [==============================] - 0s 9ms/step - loss: 1.4570 - accuracy: 0.4296 - val_loss: 1.4503 - val_accuracy: 0.5074
Epoch 8/100
9/9 [=======

In [153]:
predictions = model.predict(testX, batch_size=32)
predictions2 = predictions.argmax(axis=1)

9/9 [==============================] - 0s 1ms/step


In [154]:
testY_arg = testY.argmax(axis=1)
accuracy_score(testY_arg, predictions2)

0.8555555555555555

### Анализ нейросетью через Dimensions: сборка по половинкам корпусов новостей, эссе и рассказов

In [155]:
min_max_scaler = preprocessing.MinMaxScaler()

train_X_dim = min_max_scaler.fit_transform(X_train_dims.values)
test_X_dim = min_max_scaler.fit_transform(X_test_dims.values)

lb = LabelBinarizer()
trainY = lb.fit_transform(y_train_dims)
testY = lb.transform(y_test_dims)

model = Sequential()
model.add(Dense(1024, input_shape=(6,), activation="sigmoid"))
model.add(Dense(512, activation="sigmoid"))
model.add(Dense(len(lb.classes_), activation="softmax"))

model.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

H = model.fit(train_X_dim, trainY, validation_data=(test_X_dim, testY),
    epochs=100, batch_size=32) 

predictions = model.predict(test_X_dim, batch_size=32)
predictions2 = predictions.argmax(axis=1)

testY_arg = testY.argmax(axis=1)
accuracy_score(testY_arg, predictions2)

Epoch 1/100
9/9 [==============================] - 1s 22ms/step - loss: 1.7148 - accuracy: 0.3494 - val_loss: 1.5819 - val_accuracy: 0.4428
Epoch 2/100
9/9 [==============================] - 0s 9ms/step - loss: 1.6375 - accuracy: 0.3271 - val_loss: 1.6215 - val_accuracy: 0.4428
Epoch 3/100
9/9 [==============================] - 0s 9ms/step - loss: 1.5867 - accuracy: 0.4201 - val_loss: 1.4858 - val_accuracy: 0.5424
Epoch 4/100
9/9 [==============================] - 0s 9ms/step - loss: 1.5304 - accuracy: 0.4275 - val_loss: 1.4818 - val_accuracy: 0.4428
Epoch 5/100
9/9 [==============================] - 0s 9ms/step - loss: 1.5030 - accuracy: 0.4201 - val_loss: 1.4516 - val_accuracy: 0.4428
Epoch 6/100
9/9 [==============================] - 0s 9ms/step - loss: 1.4507 - accuracy: 0.4201 - val_loss: 1.4377 - val_accuracy: 0.4428
Epoch 7/100
9/9 [==============================] - 0s 9ms/step - loss: 1.4721 - accuracy: 0.4201 - val_loss: 1.4043 - val_accuracy: 0.4428
Epoch 8/100
9/9 [=========

0.7490774907749077

# ----------------------------------------------------------------------------------------